In [3]:
# make detail results dict
import json
import jsonlines
import re
import string
def load_ground_truths(
    ground_truth_file_path: str,
):
    id_to_ground_truths = {}
    with jsonlines.open(ground_truth_file_path, 'r') as input_file:
        for line in input_file:
            #import pdb; 
            qid = line['question_id']
            answer = line['answers_objects'][0]['spans']
            id_to_ground_truths[qid] = answer
    return id_to_ground_truths
def load_predictions(prediction_file_path):
    with open(prediction_file_path, "r") as file:
        id_to_predictions = json.load(file)
    return id_to_predictions

def answer_extractor(potentially_cot: str) -> str:
    # In a few experiments I forgot the configuring the answer extractor part
    # and so the final answer is a cot chain instead. Instead of having to do
    # all those exps again, I'm just doing answer_extraction here. This needs
    # to be fixed later though.

    if potentially_cot.startswith('"') and potentially_cot.endswith('"'):
        potentially_cot = potentially_cot[1:-1]

    cot_regex = re.compile(".* answer is:? (.*)\\.?")
    match = cot_regex.match(potentially_cot)
    if match:
        output = match.group(1)
        if output.endswith("."):
            output = output[:-1]
    else:
        output = potentially_cot

    return output

def calculate_acc(prediction, ground_truth):
    for gt in ground_truth:
        if gt in prediction:
            return 1
    return 0

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s)))) 


In [11]:
from official_evaluation.wikimultihopqa2.wikimultihop_evaluate import my_eval

#{qid:{"one":{f1:"",acc:""}}
base_path = '/apdcephfs/private_zivtang/code/Adaptive-RAG/predictions/dev_500/ircot_qa_flan_t5_xl_2wikimultihopqa____prompt_set_1___bm25_retrieval_count__6___distractor_count__1/'
data_name = '2wikimultihopqa'
# # read prediction__2wikimultihopqa_to_2wikimultihopqa__dev_500_subsampled.json
# with open(base_path+'prediction__2wikimultihopqa_to_2wikimultihopqa__dev_500_subsampled.json', 'r') as f:
#     prediction = json.load(f)

id_to_predictions = load_predictions(base_path+'prediction__2wikimultihopqa_to_2wikimultihopqa__dev_500_subsampled.json')

id_to_ground_truths = load_ground_truths('processed_data/'+data_name+'/dev_500_subsampled.jsonl')

# load /apdcephfs/private_zivtang/code/Adaptive-RAG/raw_data/2wikimultihopqa/id_aliases.json
aliases= {}
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/raw_data/2wikimultihopqa/id_aliases.json') as f:
    for json_line in map(json.loads, f):
        aliases[json_line["Q_id"]] = {
            "aliases": set(json_line["aliases"] + json_line["demonyms"])
        }

for id_ in set(id_to_ground_truths.keys()):
    ground_truth = id_to_ground_truths[id_]
    prediction = id_to_predictions[id_]

